Data Augmentation : EDA:SR

# **setting**

In [21]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

#import os
#os.chdir('./data')
#os.getcwd()

In [22]:
def preprocessing_dataset(dataset):
    """처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []

    for i, j in tqdm(zip(dataset["subject_entity"], dataset["object_entity"]), desc="preprocessing"):
        i = i[1:-1].split(",")[0].split(":")[1]
        j = j[1:-1].split(",")[0].split(":")[1]

        subject_entity.append(i)
        object_entity.append(j)

    out_dataset = pd.DataFrame(
        {
            "id": dataset["id"],
            "sentence": dataset["sentence"],
            "subject_entity": subject_entity,
            "object_entity": object_entity,
            "label": dataset["label"],
        }
    )
    return out_dataset


def load_data(dataset_dir):
    """csv 파일을 경로에 맡게 불러 옵니다."""
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)

    return dataset

def label_to_num(label):
    num_label = []
    with open("/opt/ml/code/dict_label_to_num.pkl", "rb") as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label

# **데이터 불러오기**

In [23]:
train_path = '/opt/ml/dataset/train/train.csv'
total_data = load_data(train_path)
# total_data.columns

preprocessing: 32470it [00:00, 371483.11it/s]


In [24]:
train_data = total_data.sample(frac=0.9, random_state=42)
val_data = total_data.drop(train_data.index)

train_label = label_to_num(train_data["label"].values)
val_label = label_to_num(val_data["label"].values)

In [76]:
pd.options.display.max_colwidth = 2000

# **SR**   
- pororo를 활용한 유의어로 대체
- sub,obj entity 단어를 비슷한 의미의 유의어로 교체

### **Base Code**

In [23]:
#pip install pororo
from pororo import Pororo

pg = Pororo(task="pg", lang="ko")

def pororo_pg(faq_question):
    question_based_q = pg(faq_question)
    return question_based_q

In [11]:
sentence = "결제가 안되는데 무엇을 어떻게 해야되나요?"
pg(sentence)

'결제가 안 되는데 무엇을 하면 좋을까요?'

### **TEST**

In [41]:
train_data[:10].sentence

2893                                                                                                                           문하시중 이자연의 조카이고 인예왕후, 인경현비, 인절현비와는 사촌간이다.
25844                                                                                                 1년 후, 바이에른은 전설적인 오스트리아인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다.
17759    그러다가 1949년 6월 6일 이승만 대통령과 신성모 내무부 장관의 사주를 받은 친일 경찰이 반민특위 사무실을 습격하는 사건 발생 후 반민특위가 사실상 무력화되면서 서울 마포형무소에 구금되었던 하판락은 서울에서 3회, 부산에서 1회 등 모두 4차 공판을 거쳐 최종 무혐의로 풀려났다.
10515                                                                무라트 1세의 아들 바예지드 1세는 1396년에 불가리아 북부에서 니코폴리스 전투를 벌여, 이 전투에서 헝가리 왕국을 필두로 한 십자군을 격파, 오히려 영토를 더 크게 넓혔다.
12680                                       실제로 남한 쪽에서는 남조선로동당·근로인민당 등 좌익계열 정당뿐 아니라 한국독립당·민족자주연맹 등 우익계열 정당들도 참여하였을 뿐만 아니라, 박헌영·백남운·김구·김규식·조소앙 등 명망있는 좌익 및 우익 인사들도 참석했다.
16401                                                                                                                    1983년 한국 프로야구(KBO)의 삼미에 입단하면서 장명부라는 등록명

---

# **Fill in the Blank**   
- pororo 빈칸 채우기 활용
- sub,obj entity 단어를 masking

### **Base Code**

In [86]:
from pororo import Pororo
fib= Pororo(task="fib", lang="ko")
# ko,en,ja,zh

In [66]:
# fib("손흥민은 __의 축구선수이다.")

### **TEST**

In [77]:
train_data[train_data.label =='per:origin']

,id,sentence,subject_entity,object_entity,label
25844,25844,"1년 후, 바이에른은 전설적인 오스트리아인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다.",'에른스트 하펠','오스트리아',per:origin
9773,9773,마닐라에서 야당 정치인 베니그노 아키노와 필리핀 11대 대통령 코라손 아키노 사이의 1남 4녀 중 셋째로 태어났다.,'코라손 아키노','필리핀',per:origin
4845,4845,"이날 행사에는 김승남지역위원장을 비롯 우원식(전)원내대표, 송갑석광주시당위원장, 김철우보성군수, 이승옥강진군수, 박병종(전)고흥군수, 김성(전)장흥군수와 4개지역 더불어민주당 소속 도의원 9명, 군의원 31명을 포함 당직자 및 당원 3,500여명이 참석하여 성황리에 개최됐다.",'우원식','원내대표',per:origin
18461,18461,메수드 1세는 39년 동안 군주로 군림하여 룸 셀주크 군주들 가운데 가장 오랜 기간 통치한 술탄이 되었다.,'메수드 1세','룸 셀주크',per:origin
30566,30566,"김명수(金命洙, 1959년 10월 12일 ~)는 대한민국의 제16대 대법원장이다.",'김명수','대한민국',per:origin
...,...,...,...,...,...
27177,27177,"해주 최씨로, 가수 최성수, 탤런트 최불암, 최수종, 최경록 전 육군참모총장하고 문중이다.",'최불암','해주 최씨',per:origin
21223,21223,"위르비 에마뉘엘손(1986년 6월 16일 ~)은 남아메리카 수리남 계통의 집안에서 태어난 네덜란드 국적의 축구 선수로, 2017년 현재 에레디비시의 FC 위트레흐트에서 뛰고 있다.",'위르비 에마뉘엘손','네덜란드',per:origin
18677,18677,또한 캐나다의 추수감사절에 대한 기원은 종종 17세기 초 탐험가 사무엘 드 샹플랭과 함께 뉴프랑스에 온 프랑스 개척민들이 성공적인 수확을 경축한 행사로까지 거슬러 올라가기도 한다.,'사무엘 드 샹플랭','뉴프랑스',per:origin
1549,1549,방패 속 얼굴 주인공은 그리스 신화의 ‘제우스’에 해당하는 '유피테르'다.,'제우스','그리스',per:origin


In [78]:
text = '1년 후, 바이에른은 전설적인 오스트리아인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다.'
s = '에른스트 하펠'
o = '오스트리아'
text_s = text.replace(s,"__")
text_o = text.replace(o,"__")

In [82]:
fib(text_s)

['들', '님', '대행', '만', '팀', '감독', '직', '등', ')', '급', '경력', '본인', '단장']